In [ ]:
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


In [ ]:
import surp
from surp import gce_math as gcem
import arya

from surp import subgiants

In [ ]:
data_dir = surp._globals.DATA_DIR

In [ ]:
def keil_plot(teff, logg, bins=None, **kwargs):
    if bins is None:
        bins = 200
        xbins = np.linspace(np.nanmin(teff), np.nanmax(teff), bins)
        ybins = np.linspace(np.nanmin(logg), np.nanmax(logg), bins)
        
    plt.scatter(teff, logg, **kwargs)

    ax = plt.gca()
    if not ax.xaxis_inverted():
        plt.gca().invert_xaxis()
        plt.gca().invert_yaxis()
    plt.xlabel(r"$T_{\rm eff}$ / K")
    plt.ylabel(r"$\log g$")

In [ ]:
def plot_ab(df, x, y, **kwargs):
    plt.scatter(df[x], df[y], **kwargs)
    #arya.plotting.medianplot(df, x, y, numbins=25, color="k")
    plt.xlabel(to_nice_name(x))
    plt.ylabel(to_nice_name(y))

In [ ]:
def to_nice_name(apogee_name):
    name = apogee_name.replace("_", "/").title()
    name = f"[{name}]"
    return name

In [ ]:
import stelllab

# Stellab

# Hypatia

In [ ]:
hypatia = Table.read(data_dir + "hypatia.fits", format="fits")

hypatia_new = pd.read_csv(data_dir + "hypatia-20052024.csv")

In [ ]:
hypatia_new

In [ ]:
filt = hypatia_new.f_logg >= 3.5
filt &= ~np.isnan(hypatia_new.C)
filt &= ~np.isnan(hypatia_new.Mg)

hypatia_clean = hypatia_new[filt].copy()

In [ ]:
hypatia_clean["C_MG"] = hypatia_clean.C - hypatia_clean.Mg
hypatia_clean["C_O"] = hypatia_clean.C - hypatia_clean.O

hypatia_clean["MG_H"] = hypatia_clean.Mg
hypatia_clean["O_H"] = hypatia_clean.O

hypatia_clean["MG_FE"] = hypatia_clean.Mg - hypatia_clean.Fe

hypatia_clean["FE_H"] = hypatia_clean.Fe
hypatia_clean["O_MG"] = hypatia_clean.O - hypatia_clean.Mg
hypatia_clean["O_FE"] = hypatia_clean.O - hypatia_clean.Fe

In [ ]:
keil_plot(hypatia_new.f_teff, hypatia_new.f_logg,alpha=0.1)
keil_plot(hypatia_clean.f_teff, hypatia_clean.f_logg,alpha=0.1)


In [ ]:
for ele1, ele2 in [("MG_H", "C_MG"), ("MG_FE", "C_MG"), ("FE_H", "MG_FE"), ( "MG_H", "O_MG",)]:
    plot_ab(hypatia_clean, ele1, ele2, alpha=0.05, s=1)
    arya.medianplot(hypatia_clean, ele1, ele2, color="k", binsize=300)
    plt.xlim(-1, 0.5)
    plt.ylim(-0.5, 0.5)
    plt.show()

# Luck 2018

In [ ]:
luck18 = Table.read(data_dir + "luck2018.fits", format="fits")
luck18 = luck18.to_pandas()

Some important columns
- C538.2: Log carbon abundance from C I 538.2 nm line
- C2: Log carbon abundance from C2 Swan lines (primary indicator at 513.5 nm)
- O615.5:  Log oxygen abundance from O I 615.5 nm triplet
- O630.0:  Log oxygen abundance from [O I] 630.0 nm line

use Cmean and Omean for 12 + log abund

In [ ]:
plt.scatter(luck18["C505_2"], luck18.C538_2 - luck18.C505_2)
plt.axhline(0)

In [ ]:
plt.scatter(luck18["O615_5"], luck18.O630_0 - luck18.O615_5)
plt.axhline(0)

In [ ]:
luck18["C_H"] = gcem.eps_to_brak(luck18["Cmean"], "C")
luck18["O_H"] = gcem.eps_to_brak(luck18["Omean"], "o")

luck18["C_O"] = luck18.C_H - luck18.O_H

luck18["FE_H"] = luck18["__Fe_H_"]
luck18["O_FE"] = luck18.O_H - luck18.FE_H


In [ ]:
plt.scatter(luck18.C_H, luck18.C_H - luck18["__C_H_mean"])

In [ ]:
plt.scatter(luck18.O_H, luck18.O_H - luck18["__O_H_mean"])

In [ ]:
keil_plot(luck18.Teff, luck18.logg)

In [ ]:
for ele1, ele2 in [("O_H", "C_O"), ("O_FE", "C_O"), ("FE_H", "O_FE")]:
    plot_ab(luck18, ele1, ele2, alpha=0.05, s=1)
    arya.medianplot(luck18, ele1, ele2, color="k", binsize=300)
    plt.xlim(-1, 0.5)
    plt.ylim(-0.5, 0.5)
    plt.show()

# HARPS GTO

delgado mena et al. (2021), https://ui.adsabs.harvard.edu/abs/2021A%26A...655A..99D/abstract 


Uses the high resolution HARPS-GTO spectrograph (R ~ 115 000).
Determines C from two atomic lines using LTE EW analysis.

- Wavelength range: 3770–6900
- SNR > 100 or > 300 
- C from 5052 Å and 5380

Oxygen was derived in Bertran de Lis et al. (2015, https://ui.adsabs.harvard.edu/abs/2015A%26A...576A..89B/abstract) from [O I] at 6158 and OI at 6300, which are labeled at O1 and O2 respectively.

In [ ]:
harps = pd.read_csv(surp.DATA_DIR + "dm21.csv", delimiter="\t *", comment="#", engine="python")

In [ ]:
keil_plot(harps.Teff, harps.logg)

In [ ]:
plt.scatter(harps.__O1_Fe_, harps.__O2_Fe_ - harps.__O1_Fe_)

In [ ]:
harps["FE_H"] = harps.__Fe_H_
harps["O_FE"] = np.nanmean([harps.__O1_Fe_, harps.__O2_Fe_], axis=0)
harps["O_H"] = harps.O_FE + harps.FE_H

harps["C_O"] = harps["__C_Fe_"] - harps.O_FE

In [ ]:
for ele1, ele2 in [("O_H", "C_O"), ("O_FE", "C_O"), ("FE_H", "O_FE")]:
    plot_ab(harps, ele1, ele2, alpha=0.1, s=3)
    arya.medianplot(harps, ele1, ele2, color="k", binsize=300)
    plt.xlim(-1, 0.5)
    plt.ylim(-0.5, 0.5)
    plt.show()

# HARPS CH
Also there is another study using HARPS GTO but with CH
https://ui.adsabs.harvard.edu/abs/2017A%26A...599A..96S/abstract
TODO: Need to combine with other data...

In [ ]:
harps_ch = Table.read(data_dir + "harps_ch.fits", format="fits")


In [ ]:

harps_ch

# Brewer + 2016

using HIRIS (R≈70,000).
LTE assumed. Our line list contains 63 atomic carbon lines; typically, these are shallow or blended

In [ ]:
brewer16 = Table.read(data_dir + "brewer+2016.fits", format="fits")
brewer16 = brewer16.to_pandas()

In [ ]:
brewer16.columns.to_list()

In [ ]:
brewer16["C_H"] = brewer16.__C_H_
brewer16["O_H"] = brewer16.__O_H_
brewer16["FE_H"] = brewer16.__Fe_H_
brewer16["MG_H"] = brewer16.__Mg_H_


brewer16["C_MG"] = brewer16.C_H - brewer16.MG_H
brewer16["MG_FE"] = brewer16.MG_H - brewer16.FE_H
brewer16["O_MG"] = brewer16.O_H - brewer16.MG_H

brewer16["C_O"] = brewer16.C_H - brewer16.O_H
brewer16["O_FE"] = brewer16.O_H - brewer16.FE_H


In [ ]:
filt = brewer16.logg > 3.5

brewer16_clean = brewer16[filt].copy()

In [ ]:
keil_plot(brewer16.Teff, brewer16.logg, alpha=0.2)
keil_plot(brewer16_clean.Teff, brewer16_clean.logg, alpha=0.2)

In [ ]:
for ele1, ele2 in [("MG_H", "C_MG"), ("MG_FE", "C_MG"), ("FE_H", "MG_FE"), ( "MG_H", "O_MG",)]:
    plot_ab(brewer16_clean, ele1, ele2, alpha=0.05, s=1)
    arya.medianplot(brewer16_clean, ele1, ele2, color="k", binsize=300)
    plt.xlim(-1, 0.5)
    plt.ylim(-0.5, 0.5)
    plt.show()

# ALL

In [ ]:
hypatia_clean.plot_kwargs = dict(
    color=arya.COLORS[1],
    binsize=300,
    label="hypatia",
    marker="s"
)

In [ ]:
brewer16_clean.plot_kwargs = dict(
    color=arya.COLORS[2],
    binsize=100,
    label="brewer"
)

In [ ]:
harps.plot_kwargs = dict(
    color=arya.COLORS[3],
    binsize=100,
    label="harps",
    marker="o"
)

In [ ]:
luck18.plot_kwargs = dict(
    color=arya.COLORS[4],
    binsize=100,
    label="luck",
    marker="*"
)

In [ ]:
studies = {
    "hypatia": hypatia_clean,
    "brewer+16": brewer16_clean,
    "harps-GTO": harps,
    "luck18": luck18
}

In [ ]:

fig, axs = plt.subplots(1, 2, figsize=(7, 3), sharey=True, gridspec_kw={"wspace": 0}, dpi=250)

plt.sca(axs[0])
x = "O_H"
y = "C_O"

i = 0

for key, study in studies.items():
    study = studies[key]
    arya.medianplot(study, x, y, aes="line", errorbar=None, **study.plot_kwargs)


plt.ylim(-0.25)

plt.xlabel("[O/H]")
plt.ylabel("[C/O]")

plt.sca(axs[1])

x = "O_FE"
y = "C_O"

i = 0

for key, study in studies.items():
    study = studies[key]
    arya.medianplot(study, x, y, aes="line", errorbar=None, **study.plot_kwargs)


plt.ylim(-0.3, 0.2)

arya.Legend()
plt.xlabel("[O/Fe]")

plt.savefig("caah_catalogues.pdf")